# Web Page Summarization Evaluation System

This notebook evaluates the quality of web page summaries generated by different LLM engines using an LLM-as-a-judge approach with G-Eval methodology.

## 1. Creating a Gold Standard

This section creates a gold standard dataset using the GPT-5.2 model to generate high-quality summaries. The gold standard will serve as a reference for evaluating other summarization engines and for distillation tasks.

In [1]:
import json
import asyncio
from agents.summarizer import Summarizer
from tqdm.asyncio import tqdm as atqdm

# Load baseline data
with open('data/baseline_1k.json', 'r', encoding='utf-8') as f:
    baseline_data = json.load(f)

# Extract the data list from the top-level structure
baseline_data = baseline_data['data']
print(f"Loaded {len(baseline_data)} items from baseline dataset")

# Initialize the gold standard summarizer with GPT-5.2
gold_summarizer = Summarizer(model="gpt-5.2-2025-12-11")

# Initialize variables to track total cost and skipped items
total_cost = 0.0
gold_standard_data = []
skipped_items = []

async def summarize_item(item):
    """Async wrapper to summarize a single item"""
    loop = asyncio.get_event_loop()
    # Run the synchronous summarize in a thread pool
    summary, cost = await loop.run_in_executor(
        None, 
        lambda: gold_summarizer.summarize(item['markdown_content'], get_cost=True)
    )
    
    # Check if the request was skipped due to token limits
    if summary is None:
        return None, item['url']
    
    return {
        'url': item['url'],
        'markdown_content': item['markdown_content'],
        'summary': summary
    }, cost

async def generate_summaries():
    """Generate all summaries concurrently"""
    global total_cost, gold_standard_data, skipped_items
    
    # Create tasks for all items (limit concurrency to avoid rate limits)
    max_concurrent = 10  # Adjust based on your API rate limits
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def summarize_with_semaphore(item):
        async with semaphore:
            return await summarize_item(item)
    
    # Process all items concurrently with progress bar
    print("Generating gold standard summaries with GPT-5.2...")
    tasks = [summarize_with_semaphore(item) for item in baseline_data]
    results = await atqdm.gather(*tasks, desc="Processing items")
    
    # Collect results and accumulate costs
    for result in results:
        if result[0] is None:
            # Item was skipped due to token limits
            skipped_items.append(result[1])
        else:
            gold_item, cost = result
            gold_standard_data.append(gold_item)
            total_cost += cost

# Run the async function
await generate_summaries()

# Save the gold standard dataset
output_path = 'data/goldstandard_1k.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(gold_standard_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Gold standard dataset saved to {output_path}")
print(f"✓ Total items processed: {len(gold_standard_data)}")
if skipped_items:
    print(f"⚠ Skipped {len(skipped_items)} items due to token limits:")
    for url in skipped_items:
        print(f"  - {url}")
print(f"✓ Total estimated cost for gold standard generation (based on tokens and rates): ${total_cost:.4f}")

Loaded 1000 items from baseline dataset
Generating gold standard summaries with GPT-5.2...


Processing items:   2%|▏         | 22/1000 [00:22<14:58,  1.09it/s] 

Processing items:  13%|█▎        | 132/1000 [01:58<13:05,  1.10it/s]

Processing items:  18%|█▊        | 183/1000 [02:45<09:00,  1.51it/s]

Processing items:  44%|████▍     | 439/1000 [06:43<08:07,  1.15it/s]

...

Processing items:  52%|█████▏    | 518/1000 [07:57<04:52,  1.65it/s]

Processing items:  65%|██████▍   | 646/1000 [09:54<04:44,  1.24it/s]

Processing items:  79%|███████▉  | 789/1000 [11:57<01:29,  2.35it/s]

Processing items: 100%|██████████| 1000/1000 [15:18<00:00,  1.09it/s]



✓ Gold standard dataset saved to data/goldstandard_1k.json
✓ Total items processed: 992
⚠ Skipped 8 items due to token limits:
  - https://pmc.ncbi.nlm.nih.gov/articles/PMC7271218/
  - https://www.jsog.or.jp/activity/pdf/gl_fujinka_2023.pdf
  - https://weatherspark.com/h/y/557/2024/Historical-Weather-during-2024-in-San-Francisco-California-United-States
  - https://servicehub.ucdavis.edu/servicehub?id=ucd_kb_article&sys_id=cf60ebc293f1e69083cc38797bba1020
  - https://s5.static.brasilescola.uol.com.br/vestibular/2022/12/resultado-cederj-2023.pdf
  - https://colab.research.google.com/github/hc9903/deepke/blob/master/isa.ipynb
  - https://www.insp.mx/resources/images/stories/INSP/Docs/Transparencia/EDICION%202011%20MEDICAMENTOS%20-%20link.pdf
  - https://s2.static.brasilescola.uol.com.br/vestibular/2024/01/resultado-cederj-2024.pdf
✓ Total estimated cost for gold standard generation (based on tokens and rates): $30.9979


For benchmarking, ignoring the long requests is fine. Production will need to handle inference that can handle these requests by cleaning / splitting the requests.

At this point I'll split this gold standard to train-validation subsets, so that all evaluations will be done on the same web-pages.

In [2]:
import json
import tiktoken

PAGE_MAX_TOKENS = 64000

# Load the gold standard dataset
with open('data/goldstandard_1k.json', 'r', encoding='utf-8') as f:
    gold_standard = json.load(f)

print(f"Loaded {len(gold_standard)} items from gold standard")

# Initialize tokenizer for token counting (using gpt-4.1 family)
tokenizer = tiktoken.encoding_for_model("gpt-4o")  # gpt-4.1 uses same tokenizer as gpt-4o
print(f"Using tokenizer: {tokenizer.name}")

# Split data based on token count
items_under_limit = []
items_over_limit = []

for item in gold_standard:
    token_count = len(tokenizer.encode(item['markdown_content']))
    if token_count <= PAGE_MAX_TOKENS:
        items_under_limit.append(item)
    else:
        items_over_limit.append(item)

print(f"\nItems under {PAGE_MAX_TOKENS:,} tokens: {len(items_under_limit)}")
print(f"Items over {PAGE_MAX_TOKENS:,} tokens: {len(items_over_limit)}")

# Calculate target training size (50% of total gold standard)
target_train_size = len(gold_standard) // 2

# Take up to 50% of total for training (only from items under limit)
train_data = items_under_limit[:target_train_size]

# Everything else goes to validation
validation_data = items_under_limit[target_train_size:] + items_over_limit

print(f"\nTrain set: {len(train_data)} items (all under token limit)")
print(f"Validation set: {len(validation_data)} items ({len(items_under_limit[target_train_size:])} under limit, {len(items_over_limit)} over limit)")

# Save train set (for distillation)
train_path = 'data/goldstandard_train.json'
with open(train_path, 'w', encoding='utf-8') as f:
    json.dump(train_data, f, indent=2, ensure_ascii=False)

# Save validation set (for evaluation)
validation_path = 'data/goldstandard_validation.json'
with open(validation_path, 'w', encoding='utf-8') as f:
    json.dump(validation_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Train set saved to {train_path}")
print(f"✓ Validation set saved to {validation_path}")
print(f"✓ All training examples are within the {PAGE_MAX_TOKENS:,} token limit for fine-tuning")

Loaded 992 items from gold standard
Using tokenizer: o200k_base

Items under 64,000 tokens: 948
Items over 64,000 tokens: 44

Train set: 496 items (all under token limit)
Validation set: 496 items (452 under limit, 44 over limit)

✓ Train set saved to data/goldstandard_train.json
✓ Validation set saved to data/goldstandard_validation.json
✓ All training examples are within the 64,000 token limit for fine-tuning


## 2. Distilling the Gold Standard into a Smaller Model

This part will attampt to distil the intelligence of the GPT-5.2 model into smaller variants of the GPT-4.1 family, in order to create faster-cheaper solutions of similar performances

In [ ]:
from train.finetune import prepare_and_train

# Fine-tune both models using the training data
models_to_train = [
    "gpt-4.1-mini-2025-04-14",
    "gpt-4.1-nano-2025-04-14"
]

finetuned_models = {}

for model in models_to_train:
    print(f"\n{'='*70}")
    print(f"Starting fine-tuning for {model}")
    print(f"{'='*70}\n")
    
    finetuned_model_id = prepare_and_train(
        model=model,
        train_json_path='data/goldstandard_train.json',
        n_epochs=3
    )
    
    finetuned_models[model] = finetuned_model_id
    print(f"\n✓ {model} → {finetuned_model_id}\n")

print(f"\n{'='*70}")
print("All fine-tuning jobs completed!")
print(f"{'='*70}")
for base, ft in finetuned_models.items():
    print(f"{base}")
    print(f"  → {ft}")



Starting fine-tuning for gpt-4.1-mini-2025-04-14


Model: gpt-4.1-mini-2025-04-14
Training samples: 496
Total tokens: 4,987,780
Epochs: 3
Estimated training cost: $74.82
Training file: data\train_gpt-4.1-mini-2025-04-14.jsonl

Fine-tuning job submitted: ftjob-GVcXQzcPFNU9KFyOIYQ1r7xK
Monitor at: https://platform.openai.com/finetune/ftjob-GVcXQzcPFNU9KFyOIYQ1r7xK

[1768775760] Created fine-tuning job: ftjob-GVcXQzcPFNU9KFyOIYQ1r7xK
[1768775760] Validating training file: file-JjnTJVi1VoX4zQpeAdRhL9
[1768775983] Files validated, moving job to queued state
[1768775986] Fine-tuning job started
[1768776107] Step 1/1488: training loss=1.07
[1768776109] Step 2/1488: training loss=1.35
[1768776111] Step 3/1488: training loss=2.69
[1768776113] Step 4/1488: training loss=1.95
[1768776115] Step 5/1488: training loss=1.75
[1768776117] Step 6/1488: training loss=1.14
[1768776119] Step 7/1488: training loss=1.55
[1768776122] Step 8/1488: training loss=1.86
[1768776122] Step 9/1488: training loss=1.1

## Benchmarking

At this point I'll define a small benchmark of 20% of the validation set for model selection, for cost and time efficiency.

In [1]:
import json
import tiktoken

PAGE_MAX_TOKENS = 250000

# Load the gold standard validation dataset
with open('data/goldstandard_validation.json', 'r', encoding='utf-8') as f:
    gold_standard = json.load(f)

print(f"Loaded {len(gold_standard)} items from gold standard validation set")

# Initialize tokenizer for token counting (using gpt-4.1 family)
tokenizer = tiktoken.encoding_for_model("gpt-4o")  # gpt-4.1 uses same tokenizer as gpt-4o
print(f"Using tokenizer: {tokenizer.name}")

# Split data based on token count
items_under_limit = []
items_over_limit = []

for item in gold_standard:
    token_count = len(tokenizer.encode(item['markdown_content']))
    if token_count <= PAGE_MAX_TOKENS:
        items_under_limit.append(item)
    else:
        items_over_limit.append(item)

print(f"\nItems under {PAGE_MAX_TOKENS:,} tokens: {len(items_under_limit)}")
print(f"Items over {PAGE_MAX_TOKENS:,} tokens: {len(items_over_limit)}")

# Calculate target benchmarking size (20% of total validation set, ~100 items)
target_size = len(gold_standard) // 5

# Take up to 20% of total for benchmarking (only from items under limit)
validation_data = items_under_limit[:target_size]

print(f"\nBenchmark set: {len(validation_data)} items (all under token limit)")

# Save benchmark set
bm_path = 'data/goldstandard_validation_benchmark.json'
with open(bm_path, 'w', encoding='utf-8') as f:
    json.dump(validation_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Benchmark set saved to {bm_path}")
print(f"✓ All benchmarking examples are within the {PAGE_MAX_TOKENS:,} token limit for fine-tuning")

# Create baseline subset matching the same URLs
print("\nCreating baseline subset with same URLs...")
with open('data/baseline_1k.json', 'r', encoding='utf-8') as f:
    baseline_full = json.load(f)

# Extract URLs from validation_data
benchmark_urls = {item['url'] for item in validation_data}

# Filter baseline data to match benchmark URLs
baseline_subset = [item for item in baseline_full['data'] if item['url'] in benchmark_urls]

print(f"Matched {len(baseline_subset)} baseline items")

# Save baseline subset
baseline_subset_path = 'data/baseline_validation_benchmark.json'
with open(baseline_subset_path, 'w', encoding='utf-8') as f:
    json.dump(baseline_subset, f, indent=2, ensure_ascii=False)

print(f"✓ Baseline subset saved to {baseline_subset_path}")


Loaded 496 items from gold standard validation set
Using tokenizer: o200k_base

Items under 250,000 tokens: 494
Items over 250,000 tokens: 2

Benchmark set: 99 items (all under token limit)

✓ Benchmark set saved to data/goldstandard_validation_benchmark.json
✓ All benchmarking examples are within the 250,000 token limit for fine-tuning

Creating baseline subset with same URLs...
Matched 99 baseline items
✓ Baseline subset saved to data/baseline_validation_benchmark.json


In [ ]:
from evaluation.benchmark import BenchmarkingSuite
from agents.config import RATES

# 1. Define models to test (from your RATES keys)
test_models = ['Baseline'] + [model for model in RATES.keys()]

# 2. Prepare subset (validation subset already defined)
subset_data = validation_data

# 3. Run flow
suite = BenchmarkingSuite(rates=RATES)
report_df = suite.run_benchmark(subset_data, test_models)

# 4. Display stylized table
report_df.set_index('model').style.background_gradient(cmap='viridis')


Model 1/10: Baseline
Loading baseline summaries from data/baseline_validation_benchmark.json
✓ Filtered to 99 baseline summaries matching eval subset
✓ Loaded 99 baseline summaries
✓ Saved to data/inference_baseline.json


[1/10] Judging:   0%|          | 0/99 [00:00<?, ?sample/s]